In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
DATASET_DIR = 'dataset'
IMG_SIZE = 64

In [5]:
def load_images_from_folder(folder_path):
    images, labels = [], []
    class_names = sorted(os.listdir(folder_path))
    label_map = {name: idx for idx, name in enumerate(class_names)}

    for label in class_names:
        class_folder = os.path.join(folder_path, label)
        if not os.path.isdir(class_folder):
            continue
        for file in os.listdir(class_folder):
            img_path = os.path.join(class_folder, file)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                images.append(img)
                labels.append(label_map[label])
            except:
                print(f"Error reading: {img_path}")

    return np.array(images), np.array(labels), label_map


In [6]:
X, y, label_map = load_images_from_folder(DATASET_DIR)


In [7]:
X = X / 255.0
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_cat = to_categorical(y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y_cat, test_size=0.2)


In [9]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_map), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Save model & label map
model.save('model/sign_model.h5')
np.save('model/label_map.npy', label_map)





Epoch 1/10


2190/2190 [==============================] - 71s 31ms/step - loss: 1.0811 - accuracy: 0.6780 - val_loss: 0.2678 - val_accuracy: 0.9222
Epoch 2/10
2190/2190 [==============================] - 69s 31ms/step - loss: 0.1986 - accuracy: 0.9362 - val_loss: 0.1226 - val_accuracy: 0.9626
Epoch 3/10
2190/2190 [==============================] - 68s 31ms/step - loss: 0.1065 - accuracy: 0.9660 - val_loss: 0.1005 - val_accuracy: 0.9688
Epoch 4/10
2190/2190 [==============================] - 67s 31ms/step - loss: 0.0697 - accuracy: 0.9784 - val_loss: 0.0737 - val_accuracy: 0.9751
Epoch 5/10
2190/2190 [==============================] - 68s 31ms/step - loss: 0.0555 - accuracy: 0.9824 - val_loss: 0.0998 - val_accuracy: 0.9753
Epoch 6/10
2190/2190 [==============================] - 68s 31ms/step - loss: 0.0475 - accuracy: 0.9859 - val_loss: 0.0660 - val_accuracy: 0.9801
Epoch 7/10
2190/2190 [==============================] - 68s 31ms/step - loss: 0.0332 - accuracy: 0.9903 - val_loss: 0.06

c:\Users\prvme\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
